In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10))
X = torch.rand(2, 20)
net(X)

tensor([[ 0.1098,  0.1838,  0.2338,  0.1001,  0.0081, -0.0566,  0.3532, -0.0348,
          0.1190,  0.0354],
        [ 0.1055,  0.2871,  0.2994, -0.0185,  0.1067, -0.1869,  0.3559,  0.0297,
          0.3899,  0.0990]], grad_fn=<AddmmBackward0>)

In [3]:
class MLP(nn.Module):

    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)

tensor([[ 0.0595,  0.0576, -0.0841, -0.0230, -0.0570,  0.1802, -0.1252, -0.0192,
         -0.1377,  0.1010],
        [-0.0842,  0.2932, -0.0165, -0.0355, -0.0566,  0.2045, -0.0050, -0.1064,
         -0.2470,  0.1011]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):

    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # module是Module子类的一个实例
            # module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.2294,  0.1788, -0.0582, -0.3000, -0.0992, -0.1116, -0.0882, -0.2225,
         -0.0287,  0.0522],
        [-0.3334,  0.0949, -0.0291, -0.2262, -0.0242, -0.0581, -0.0583, -0.2172,
         -0.0284,  0.0530]], grad_fn=<AddmmBackward0>)

In [5]:
class FixedHiddenMLP(nn.Module):

    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.4321, grad_fn=<SumBackward0>)

In [7]:
class NestMLP(nn.Module):

    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64), nn.ReLU(), nn.Linear(64,32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0849, grad_fn=<SumBackward0>)